In [1]:
import utils.conf as conf
import preprocessing.data_cleaner as cleaner
import preprocessing.feature_extractor as extractor
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
train = pd.read_csv(conf.train_data_path, sep=',',usecols=['ScanCount','VisitNumber','FinelineNumber'])

In [3]:
df = train
dummies = pd.get_dummies(df.FinelineNumber)
dummy_names = ['fn_' + str(x) for x in dummies.columns]
df[dummy_names] = dummies

In [4]:
del dummies

In [6]:
del df['FinelineNumber']

SyntaxError: invalid syntax (<ipython-input-16-abb667633aec>, line 1)

In [3]:
dummies = pd.get_dummies(train.FinelineNumber)
dummies_names = [('fn_' + str(cn)) for cn in dummies.columns]

In [50]:
mask = train.FinelineNumber.value_counts() > 1000


IndexingError: Unalignable boolean Series key provided

True

In [39]:
train.FinelineNumber.nunique()

5195

In [ ]:
train.head()

In [2]:
train,_ = cleaner.read_data()

In [33]:
train, test = cleaner.clean()

In [34]:
train = extractor.aggregate_features_v2(train)
test = extractor.aggregate_features_v2(test)


KeyError: 'Column not found: TripType'

**Cross validation**

In [39]:
X = train.drop(['TripType'], axis=1)

In [40]:
from sklearn import preprocessing

tt_le = preprocessing.LabelEncoder()
tt_le.fit(train['TripType'])

LabelEncoder()

In [41]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [42]:
vn = train['VisitNumber'].unique()

In [43]:
from sklearn.model_selection import train_test_split
train_vn, test_vn = train_test_split(vn, test_size=0.4)
train_index = train['VisitNumber'].isin(train_vn)
y_train = tt_le.transform(train[train_index]['TripType'])
X_train = train[train_index].drop('TripType', axis=1)
y_test = tt_le.transform(train[-train_index]['TripType'])
X_test = train[-train_index].drop('TripType', axis=1)

In [49]:
X_train = extractor.aggreate_features(X_train)
X_test = extractor.aggreate_features(X_test)

** add feature **
* number of returned items per trip

In [50]:
X_train.drop('VisitNumber',axis=1, inplace=True)

In [51]:
X_test.drop('VisitNumber', axis=1, inplace=True)

In [74]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, n_jobs=4)
clf.fit(X_train,y_train)
clf_probs = clf.predict_proba(X_test)

In [75]:
import numpy as np
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X_train.shape[1]):
    print("%d. feature %d : %s (%f)" % (f + 1, indices[f], X_train.columns[indices[f]],importances[indices[f]]))
#from sklearn.model_selection import cross_val_score
#scores = cross_val_score(clf, X_test, y_test.values)
scores = clf.score(X_test,y_test)
print(scores)

from sklearn.metrics import log_loss
logloss_score = log_loss(y_test, clf_probs)
print(logloss_score)

Feature ranking:
1. feature 11 : Encoded_DepartmentDescription_has_max_by_vn (0.135551)
2. feature 13 : ScanCount_purchased_items_by_vn (0.129917)
3. feature 9 : FinelineNumber_has_max_by_vn (0.124469)
4. feature 7 : Encoded_Upc_nunique_groupby_vn (0.096143)
5. feature 6 : FinelineNumber_nunique_groupby_vn (0.080364)
6. feature 10 : Encoded_DepartmentDescription_max_by_vn (0.066343)
7. feature 5 : Encoded_DepartmentDescription_nunique_groupby_vn (0.065772)
8. feature 4 : Encoded_Weekday (0.064907)
9. feature 3 : Encoded_Upc (0.060493)
10. feature 2 : Encoded_DepartmentDescription (0.057254)
11. feature 1 : FinelineNumber (0.053116)
12. feature 8 : FinelineNumber_max_by_vn (0.032754)
13. feature 12 : ScanCount_returned_items_by_vn (0.021712)
14. feature 0 : ScanCount (0.011204)
0.608704168123
2.9213188378


Feature ranking:
1. feature 11 : Encoded_DepartmentDescription_has_max_by_vn (0.135551)
2. feature 13 : ScanCount_purchased_items_by_vn (0.129917)
3. feature 9 : FinelineNumber_has_max_by_vn (0.124469)
4. feature 7 : Encoded_Upc_nunique_groupby_vn (0.096143)
5. feature 6 : FinelineNumber_nunique_groupby_vn (0.080364)
6. feature 10 : Encoded_DepartmentDescription_max_by_vn (0.066343)
7. feature 5 : Encoded_DepartmentDescription_nunique_groupby_vn (0.065772)
8. feature 4 : Encoded_Weekday (0.064907)
9. feature 3 : Encoded_Upc (0.060493)
10. feature 2 : Encoded_DepartmentDescription (0.057254)
11. feature 1 : FinelineNumber (0.053116)
12. feature 8 : FinelineNumber_max_by_vn (0.032754)
13. feature 12 : ScanCount_returned_items_by_vn (0.021712)
14. feature 0 : ScanCount (0.011204)

0.608704168123

2.9213188378

Feature ranking:
1. feature 12 : Encoded_DepartmentDescription_has_max_by_vn (0.134543)
2. feature 5 : ScanCount_sum_groupby_vn (0.128942)
3. feature 10 : FinelineNumber_has_max_by_vn (0.123942)
4. feature 8 : Encoded_Upc_nunique_groupby_vn (0.098007)
5. feature 7 : FinelineNumber_nunique_groupby_vn (0.092697)
6. feature 6 : Encoded_DepartmentDescription_nunique_groupby_vn (0.067066)
7. feature 4 : Encoded_Weekday (0.063992)
8. feature 11 : Encoded_DepartmentDescription_max_by_vn (0.060958)
9. feature 3 : Encoded_Upc (0.060370)
10. feature 2 : Encoded_DepartmentDescription (0.057118)
11. feature 1 : FinelineNumber (0.053392)
12. feature 9 : FinelineNumber_max_by_vn (0.031842)
13. feature 13 : ScanCount_returned_items_by_vn (0.017312)
14. feature 0 : ScanCount (0.009820)

0.610022798277

2.906785231


Feature ranking:
1. feature 5 : ScanCount_sum_groupby_vn (0.146234)
2. feature 12 : Encoded_DepartmentDescription_has_max_by_vn (0.136101)
3. feature 10 : FinelineNumber_has_max_by_vn (0.126257)
4. feature 8 : Encoded_Upc_nunique_groupby_vn (0.093432)
5. feature 7 : FinelineNumber_nunique_groupby_vn (0.086363)
6. feature 6 : Encoded_DepartmentDescription_nunique_groupby_vn (0.068791)
7. feature 11 : Encoded_DepartmentDescription_max_by_vn (0.065487)
8. feature 4 : Encoded_Weekday (0.065123)
9. feature 3 : Encoded_Upc (0.060467)
10. feature 2 : Encoded_DepartmentDescription (0.055747)
11. feature 1 : FinelineNumber (0.052079)
12. feature 9 : FinelineNumber_max_by_vn (0.032862)
13. feature 0 : ScanCount (0.011058)
    
0.6123093149686295

2.64528643116

**Prepare for submission**

In [49]:
train = extractor.aggreate_features(train)
test = extractor.aggreate_features(test)

In [4]:
import models.models as models

y_train = train['TripType']
X_train = train.drop(['TripType'],axis=1)
rf_clf = models.RandomForestMultiClass()
rf_clf.fit(X_train, y_train)

RandomForest(n_estimators=50, n_jobs=4)

In [5]:
preds, classes = rf_clf.predict_proba(test)

In [ ]:
column_names = ['VisitNumber']
for cls in classes:
    column_names.append('TripType_' + str(cls))
    import pandas as pd
out = pd.DataFrame(columns=column_names)
out['VisitNumber'] = test['VisitNumber']
out.ix[:,'TripType_3':] = preds
out = out.groupby('VisitNumber').mean()
out.to_csv('submission.csv', sep=',')

**XGBoost** 

In [3]:
import xgboost as xgb

/home/wei/anaconda2/envs/pt27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [53]:
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)

In [54]:
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 38

In [55]:
watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist )
# get prediction
pred = bst.predict( xg_test )

[0]	train-merror:0.369525	test-merror:0.384245
[1]	train-merror:0.358301	test-merror:0.373553
[2]	train-merror:0.35488	test-merror:0.371409
[3]	train-merror:0.352767	test-merror:0.369727
[4]	train-merror:0.350143	test-merror:0.368865


In [66]:
print ('predicting, classification accuracy = %f' % (1 - sum( int(pred[i]) != y_test[i] \
                                                    for i in range(len(y_test))) / float(len(y_test))))

predicting, classification accuracy = 0.631135


In [59]:
import numpy as np
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bst = xgb.train(param, xg_train, num_round, watchlist );
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
preds = bst.predict( xg_test ).reshape( y_test.shape[0], 38 )
classes = tt_le.inverse_transform(np.argmax(yprob, axis=1))

[0]	train-merror:0.369525	test-merror:0.384245
[1]	train-merror:0.358301	test-merror:0.373553
[2]	train-merror:0.35488	test-merror:0.371409
[3]	train-merror:0.352767	test-merror:0.369727
[4]	train-merror:0.350143	test-merror:0.368865


In [60]:
print ('predicting, classification error=%f' % (sum( int(ylabel[i]) != y_test[i] \
                                                    for i in range(len(y_test))) / float(len(y_test)) ))

predicting, classification error=0.368865


In [64]:
from sklearn.metrics import log_loss
logloss_score = log_loss(y_test, yprob)
print(logloss_score)

2.17541434135


In [50]:
import models.models as models

y_train = train['TripType']
X_train = train.drop(['TripType'],axis=1)
xgb_clf = models.XGBoostMutliClass(nthread=4, eta=0.1, max_depth=6, num_round=5, silent=1)

In [51]:
xgb_clf.fit(X_train, y_train)

XGBoostMutliClass(eta=0.1, max_depth=6, nthread=4, num_round=5, silent=1)

In [52]:
preds, classes = xgb_clf.predict_proba(test)

In [28]:
len(classes)

38

In [53]:
column_names = ['VisitNumber']
for cls in classes:
    column_names.append('TripType_' + str(cls))
    import pandas as pd
out = pd.DataFrame(columns=column_names)
out['VisitNumber'] = test['VisitNumber']
out.ix[:,'TripType_3':] = preds
out = out.groupby('VisitNumber').mean()
out.to_csv('submission.csv', sep=',')

In [40]:
out

,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,TripType_15,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,0.016609,0.016708,0.017749,0.016925,0.020090,0.021104,0.025215,0.016409,0.016419,0.024079,...,0.018294,0.025937,0.045093,0.041908,0.066480,0.018876,0.027893,0.018247,0.022993,0.019770
2,0.016328,0.016478,0.017546,0.016499,0.024234,0.027917,0.021683,0.016114,0.016124,0.022713,...,0.018423,0.022766,0.024569,0.044900,0.059326,0.018403,0.022134,0.019700,0.021951,0.081519
3,0.015935,0.016109,0.020005,0.016049,0.016140,0.017845,0.031672,0.015753,0.015763,0.021214,...,0.018645,0.017571,0.018170,0.029232,0.049735,0.017347,0.031313,0.017956,0.024228,0.086215
4,0.016481,0.016769,0.017753,0.016982,0.021268,0.020194,0.018427,0.016399,0.016409,0.022628,...,0.017970,0.028469,0.024476,0.053138,0.056154,0.017018,0.019085,0.022476,0.019586,0.017487
6,0.015734,0.015617,0.017004,0.015815,0.016457,0.022935,0.017161,0.015272,0.015281,0.022259,...,0.018375,0.017806,0.017429,0.032650,0.038872,0.016770,0.025994,0.017951,0.025446,0.100742
13,0.020904,0.021003,0.022240,0.021402,0.023929,0.029972,0.035463,0.020729,0.020741,0.028242,...,0.022551,0.028392,0.028045,0.053446,0.074586,0.021305,0.023282,0.022672,0.024184,0.022105
14,0.020186,0.020408,0.021789,0.020809,0.027509,0.029049,0.028946,0.019958,0.019970,0.029051,...,0.023604,0.029160,0.027329,0.055401,0.088739,0.021555,0.031240,0.023625,0.027833,0.023276
16,0.019972,0.020299,0.022070,0.020812,0.026729,0.033858,0.022605,0.019731,0.019743,0.027389,...,0.022277,0.034575,0.031733,0.063202,0.097788,0.020550,0.022970,0.022524,0.023887,0.021114
18,0.020520,0.020593,0.022166,0.021603,0.028595,0.033420,0.024871,0.020188,0.020200,0.027769,...,0.023233,0.033922,0.031034,0.060936,0.092453,0.022126,0.026168,0.022581,0.026018,0.022094


In [8]:
import pandas as pd
df = pd.DataFrame(preds)

In [9]:
df['VisitNumber'] = test['VisitNumber']

In [11]:
df.groupby('VisitNumber').mean().to_csv('submission.csv', sep=',')

In [16]:
out.columns = column_names

ValueError: Length mismatch: Expected axis has 38 elements, new values have 653647 elements

In [22]:
len(classes)

653646